# Выгрузка данных из БД SQLite и подготовка дашборда по проекту "Анализ текущего уровня потребительской лояльности"

# Содержание
1. [Выгрузка данных.](#intro) 
2. [Предобработка данных.](#intro1) 
3. [Выгрузка данных с обновлённым запросом с последующим сохранением в файл.](#intro2)
4. [Ссылка на дашборд на сайте Tableau Public.](#intro3)
5. [Ссылка на pdf-файл с презентацией.](#intro4)

# 1. Выгрузка данных. <a id='intro'></a>

In [1]:
# импортирование библиотеки pandas
import pandas as pd

# импортирование библиотеки numpy
import numpy as np

# импортирование библиотеки sqlalchemy
from sqlalchemy import create_engine

In [2]:
# указание пути к файлу 
# с сохранением в переменную path_to_db и
# подключение к БД SQLite

path_to_db = 'C:/Users/solod/Downloads/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
# создание запроса для
# выгрузки данных из таблиц 
# в БД SQLite

query = """
WITH
i_1 AS (SELECT user_id,
               lt_day,
               CASE  
               WHEN lt_day <= 365 THEN 'True'
               WHEN lt_day > 365 THEN 'False'
               END AS is_new,
               age,
               CASE 
               WHEN gender_segment = 1 THEN 'женщина'
               WHEN gender_segment = 0 THEN 'мужчина'
               END AS gender_segment
        FROM user),
i_2 AS (SELECT user_id,
               nps_score,
               CASE
               WHEN nps_score BETWEEN 0 AND 6 THEN 'критики'
               WHEN nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
               WHEN nps_score BETWEEN 9 AND 10 THEN 'сторонники'
               END AS nps_group
        FROM user)
        
SELECT i_1.user_id,
       i_1.lt_day,
       i_1.is_new,
       i_1.age,
       i_1.gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       ag.title AS age_segment,
       ts.title AS traffic_segment,
       ls.title AS lifetime_segment,
       i_2.nps_score,
       i_2.nps_group
FROM i_1
INNER JOIN i_2 ON i_1.user_id = i_2.user_id
INNER JOIN user AS u ON i_2.user_id = u.user_id
INNER JOIN location AS l ON u.location_id = l.location_id
INNER JOIN age_segment AS ag ON u.age_gr_id = ag.age_gr_id
INNER JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
INNER JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id;
"""

In [4]:
# чтение запроса с 
# сохранением в переменную df и
# просмотр первых 10 значений

df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,False,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,False,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,False,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники
3,A004TB,4190,False,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонники
4,A004XT,1163,False,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонники
5,A005O0,5501,False,42.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,критики
6,A0061R,1236,False,45.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,сторонники
7,A009KS,313,True,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,сторонники
8,A00AES,3238,False,36.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,сторонники
9,A00F70,4479,False,54.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,сторонники


In [5]:
# получение общей информации о данных
# из таблицы df

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [6]:
# получение информации об описательных статиcтических данных
# в таблице df

df.describe()

,lt_day,age,nps_score
count,502493.000000,501939.000000,502493.000000
mean,1868.841439,39.621946,7.508562
std,1683.701762,11.188249,3.020378
min,-21.000000,10.000000,1.000000
25%,533.000000,31.000000,5.000000
50%,1239.000000,38.000000,9.000000
75%,3064.000000,47.000000,10.000000
max,9162.000000,89.000000,10.000000


По представленной информации мы видим, что пропуски в данных есть в двух столбцах, это `age` и `gender_segment`, однако, везде названия столбцов указаны верно. Далее необходимо проверить данные на явные и неявные дубликаты, а также на возможные аномалии и посмотреть, можно и нужно ли как то заполнить пропущенные значения в столбцах `age` и `gender_segment`.

# 2. Предобработка данных. <a id='intro1'></a>

In [7]:
# получение информации о наличии
# явных дубликатов

df.duplicated().sum()

0

In [8]:
# получение уникальных значений 
# столбца 'user_id'

df['user_id'].value_counts()

BAXQ67    1
A5RLV7    1
LQ6ZAM    1
GIEXE7    1
XPPM99    1
         ..
BV16KO    1
DQ8DDT    1
NY9M6V    1
G41I5Y    1
ZCVWIS    1
Name: user_id, Length: 502493, dtype: int64

In [9]:
# получение уникальных значений 
# столбца 'lt_day'

df['lt_day'].value_counts()

33      497
57      485
34      448
40      445
37      442
       ... 
6729      1
7241      1
7091      1
6835      1
7384      1
Name: lt_day, Length: 6950, dtype: int64

In [10]:
# получение уникальных значений в
# столбце 'lt_day'

df['lt_day'].value_counts()

33      497
57      485
34      448
40      445
37      442
       ... 
6729      1
7241      1
7091      1
6835      1
7384      1
Name: lt_day, Length: 6950, dtype: int64

In [11]:
# получение уникальных значений 
# столбца 'lt_day'

#sorted(df['lt_day'].unique())

In [12]:
# получение среза данных 
# по столбцу 'lt_day'

df.query('lt_day <= 0')

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
7666,AEAC2R,-8,True,NaN,None,ANDROID,SMARTPHONE,Россия,Томск,08 n/a,07 15-20,08 36+,1,критики
53997,CS0HF8,-2,True,NaN,None,ANDROID,SMARTPHONE,Россия,Ижевск,08 n/a,08 20-25,08 36+,10,сторонники
71577,DORPT2,-4,True,NaN,None,ANDROID,SMARTPHONE,Россия,Москва,08 n/a,08 20-25,08 36+,4,критики
100152,F5O3CG,-21,True,NaN,None,ANDROID,SMARTPHONE,Россия,Чита,08 n/a,11 35-40,08 36+,7,нейтралы
103223,FBC993,0,True,NaN,None,ANDROID,SMARTPHONE,Россия,Москва,08 n/a,04 1-5,01 1,5,критики
228087,LSE939,-13,True,NaN,None,ANDROID,SMARTPHONE,Россия,Краснодар,08 n/a,07 15-20,08 36+,1,критики
284966,OQO5GZ,0,True,NaN,мужчина,ANDROID,SMARTPHONE,Россия,Томск,08 n/a,06 10-15,01 1,8,нейтралы
325212,QTT7IR,-13,True,NaN,None,IOS,SMARTPHONE,Россия,Москва,08 n/a,06 10-15,08 36+,7,нейтралы
347784,RZUS6H,-6,True,NaN,None,ANDROID,SMARTPHONE,Россия,Красноярск,08 n/a,10 30-35,08 36+,3,критики
385897,TYWQW4,-12,True,NaN,None,IOS,SMARTPHONE,Россия,Новосибирск,08 n/a,04 1-5,08 36+,5,критики


Стоит отметить, что в столбце `lt_day` мы обнаружили отрицательные и нулевые значения по количеству дней жизни. Также согласно таблице, мы увидели, что у данных значений практически везде есть пропуски в столбцах `age`, `gender_segment`, `age_segment`, а в столбце `lifetime_segment` количество месяцев жизни превышает 36 месяцев или равняется 1 месяцу, что считается не совсем верным. Поэтому необходимо в запросе SQL убрать данные значения из таблицы, так как по количеству их насчитывается не так много.

In [13]:
# получение уникальных значений 
# столбца 'age'

df['age'].unique()

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., nan, 14., 71., 61.,
       16., 77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85.,
       87., 86.])

In [14]:
# получение уникальных значений 
# столбца 'age'

df['age'].value_counts()

33.0    20261
34.0    20031
35.0    19574
36.0    19353
32.0    19138
        ...  
83.0        9
85.0        2
87.0        2
86.0        2
89.0        1
Name: age, Length: 79, dtype: int64

In [15]:
# получение среза данных 
# по столбцам 'age', 'gender_segment' и
# 'age_segment'

df.query('age.isna() and gender_segment.isna() and age_segment == "08 n/a"')

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
751,A1E59W,4055,False,NaN,None,ANDROID,SMARTPHONE,Россия,Екатеринбург,08 n/a,04 1-5,08 36+,1,критики
1209,A28ZDT,4243,False,NaN,None,ANDROID,SMARTPHONE,Россия,Хабаровск,08 n/a,04 1-5,08 36+,5,критики
1321,A2GLPQ,2354,False,NaN,None,ANDROID,SMARTPHONE,Россия,Иркутск,08 n/a,06 10-15,08 36+,1,критики
2163,A41C1K,557,False,NaN,None,ANDROID,SMARTPHONE,Россия,РостовнаДону,08 n/a,04 1-5,06 13-24,3,критики
2667,A513VG,4705,False,NaN,None,IOS,SMARTPHONE,Россия,Екатеринбург,08 n/a,23 95-100,08 36+,9,сторонники
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498652,ZSL1TE,1389,False,NaN,None,ANDROID,SMARTPHONE,Россия,Калининград,08 n/a,04 1-5,08 36+,10,сторонники
498998,ZT6Z5A,1295,False,NaN,None,ANDROID,SMARTPHONE,Россия,Рязань,08 n/a,04 1-5,08 36+,5,критики
499598,ZUDYU9,520,False,NaN,None,ANDROID,SMARTPHONE,Россия,СанктПетербург,08 n/a,24 100+,06 13-24,9,сторонники
501246,ZXK9AK,3550,False,NaN,None,ANDROID,SMARTPHONE,Россия,Томск,08 n/a,04 1-5,08 36+,4,критики


In [16]:
# получение уникальных значений 
# столбца 'gender_segment'

df['gender_segment'].unique()

array(['женщина', 'мужчина', None], dtype=object)

Согласно представленным данным в столбцах `age` и `gender_segment` есть пропущенные значения, но оставим их пока, как есть.

In [17]:
# получение уникальных значений 
# столбца 'os_name'

df['os_name'].unique()

array(['ANDROID', 'IOS', 'OTHER', 'unknown', 'PROPRIETARY',
       'WINDOWS PHONE', 'SYMBIAN OS', 'BADA OS', 'WINDOWS MOBILE'],
      dtype=object)

In [18]:
# получение уникальных значений 
# столбца 'country'

df['country'].unique()

array(['Россия'], dtype=object)

In [19]:
# получение уникальных значений 
# столбца 'city'

df['city'].unique()

array(['Уфа', 'Киров', 'Москва', 'РостовнаДону', 'Рязань', 'Омск',
       'СанктПетербург', 'Волгоград', 'Тольятти', 'Казань', 'Самара',
       'Красноярск', 'Екатеринбург', 'Калуга', 'Краснодар', 'Иркутск',
       'Пермь', 'Владимир', 'Ижевск', 'Тюмень', 'Оренбург',
       'НижнийНовгород', 'Брянск', 'Челябинск', 'Астрахань', 'Сургут',
       'Тверь', 'Новосибирск', 'НабережныеЧелны', 'Махачкала', 'Воронеж',
       'Курск', 'Владивосток', 'Балашиха', 'Пенза', 'Калининград', 'Тула',
       'Саратов', 'Кемерово', 'Белгород', 'Барнаул', 'Чебоксары',
       'Архангельск', 'Томск', 'Ярославль', 'Ульяновск', 'Хабаровск',
       'Грозный', 'Ставрополь', 'Липецк', 'Новокузнецк', 'Якутск',
       'УланУдэ', 'Сочи', 'Иваново', 'НижнийТагил', 'Смоленск',
       'Волжский', 'Магнитогорск', 'Чита', 'Череповец', 'Саранск'],
      dtype=object)

In [20]:
# получение уникальных значений 
# столбца 'age_segment'

df['age_segment'].unique()

array(['05 45-54', '06 55-64', '04 35-44', '02 16-24', '03 25-34',
       '07 66 +', '01 до 16', '08 n/a'], dtype=object)

In [21]:
# получение уникальных значений 
# столбца 'traffic_segment'

df['traffic_segment'].unique()

array(['04 1-5', '08 20-25', '03 0.1-1', '05 5-10', '06 10-15',
       '13 45-50', '07 15-20', '09 25-30', '12 40-45', '19 75-80',
       '14 50-55', '21 85-90', '16 60-65', '18 70-75', '10 30-35',
       '20 80-85', '24 100+', '11 35-40', '15 55-60', '22 90-95',
       '17 65-70', '02 0.01-0.1', '23 95-100', '01 0-0.01', '01 0'],
      dtype=object)

In [22]:
# получение уникальных значений 
# столбца 'lifetime_segment'

df['lifetime_segment'].unique()

array(['08 36+', '06 13-24', '05 7-12', '04 4-6', '07 25-36', '02 2',
       '03 3', '01 1'], dtype=object)

In [23]:
# получение уникальных значений 
# столбца 'nps_score'

df['nps_score'].unique()

array([10,  6,  9,  2,  5,  8,  1,  4,  7,  3], dtype=int64)

Как мы видим, в данных отсутствуют явные и неявные дубликаты, формат данных также указан верный. Единственное есть пропущенные значения в столбцах `age` и `gender_segment`. Однако, не будем пока ничего менять и оставим как есть. Также мы заметили в столбце `lt_day` отрицательные и нулевые значения, поэтому далее необходимо обновить запрос по выгрузке данных и убрать подобные значения в столбце `lt_day`, а в столбцах `age_segment`, `traffic_segment` и `lifetime_segment` убрать номера, которые обозначают группы, и немного мешают восприятию других цифр, после сохранить выгруженные данные в файл для дальнейшего построения дашборда.

# 3. Выгрузка данных с обновлённым запросом с последующим сохранением в файл. <a id='intro2'></a>

In [24]:
# обновление запроса для
# выгрузки данных из таблиц 
# в БД SQLite

query = """
WITH
i_1 AS (SELECT user_id,
               lt_day,
               CASE  
               WHEN lt_day <= 365 THEN 'True'
               WHEN lt_day > 365 THEN 'False'
               END AS is_new,
               age,
               CASE 
               WHEN gender_segment = 1 THEN 'женщина'
               WHEN gender_segment = 0 THEN 'мужчина'
               END AS gender_segment
        FROM user),
i_2 AS (SELECT user_id,
               nps_score,
               CASE
               WHEN nps_score BETWEEN 0 AND 6 THEN 'критики'
               WHEN nps_score BETWEEN 7 AND 8 THEN 'нейтралы'
               WHEN nps_score BETWEEN 9 AND 10 THEN 'сторонники'
               END AS nps_group
        FROM user)
        
SELECT i_1.user_id,
       i_1.lt_day,
       i_1.is_new,
       i_1.age,
       i_1.gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       substr(ag.title, 4) AS age_segment,
       substr(ts.title, 4) AS traffic_segment,
       substr(ls.title, 4) AS lifetime_segment,
       i_2.nps_score,
       i_2.nps_group
FROM i_1
INNER JOIN i_2 ON i_1.user_id = i_2.user_id
INNER JOIN user AS u ON i_2.user_id = u.user_id
INNER JOIN location AS l ON u.location_id = l.location_id
INNER JOIN age_segment AS ag ON u.age_gr_id = ag.age_gr_id
INNER JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
INNER JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id
WHERE i_1.lt_day > 0;
"""

In [25]:
# чтение запроса с 
# сохранением в переменную df и
# просмотр первых 10 значений

df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,False,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,False,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,False,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,False,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,False,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники
5,A005O0,5501,False,42.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,False,45.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,сторонники
7,A009KS,313,True,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,сторонники
8,A00AES,3238,False,36.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,сторонники
9,A00F70,4479,False,54.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,сторонники


In [26]:
# получение общей информации о данных
# из таблицы df

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502476 entries, 0 to 502475
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502476 non-null  object 
 1   lt_day            502476 non-null  int64  
 2   is_new            502476 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501190 non-null  object 
 5   os_name           502476 non-null  object 
 6   cpe_type_name     502476 non-null  object 
 7   country           502476 non-null  object 
 8   city              502476 non-null  object 
 9   age_segment       502476 non-null  object 
 10  traffic_segment   502476 non-null  object 
 11  lifetime_segment  502476 non-null  object 
 12  nps_score         502476 non-null  int64  
 13  nps_group         502476 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [27]:
# сохранение таблицы df в 
# файл 'telecomm_csi_tableau.csv'

df.to_csv('telecomm_csi_tableau.csv', index=False)

# 4. Ссылка на дашборд на сайте Tableau Public: <a id='intro3'></a>

https://public.tableau.com/app/profile/maya4053/viz/_16502927414490/sheet0_1?publish=yes

# 5. Ссылка на pdf-файл с презентацией: <a id='intro4'></a>

https://drive.google.com/file/d/1zSNvFYOU6eKtlWApLghDpwKAcf-P9b3x/view?usp=sharing